In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import pandas as pd

# 定义模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.AvgPool2d(kernel_size=2)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.AvgPool2d(kernel_size=2)

        self.flatten = nn.Flatten()

        # 计算池化后的尺寸（假设输入图像尺寸为224x224）
        # 经过五次2x2的池化操作后，图像尺寸为 (224 / 2 / 2 / 2 / 2 / 2) = 7
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# 初始化模型
model = CNN()

# 加载模型权重，并调整为单GPU或CPU格式
def load_model(model, model_path):
    # 加载保存的状态字典
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    # 加载调整后的状态字典
    model.load_state_dict(state_dict)

# 预测函数
def predict_image(model, image_path):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
    return output.item()

# 模型参数路径
model_paths = [
    r"E:\pic_diffusion_vae\predictmodel/CNN_parametersd33Best.pth",
    r"E:\pic_diffusion_vae\predictmodel/CNN_parametersjiedianBest.pth",
    r"E:\pic_diffusion_vae\predictmodel/CNN_parametersC33Best.pth"
]

# 图片路径和名称
image_dir = r"E:\picture_diffusion_vae1\cat540_320_1.33_0.208/"
image_names = [f"lcy{i}.jpg" for i in range(1, 1000)]  # 假设图片格式为 jpg
results = []

for image_name in image_names:
    image_path = os.path.join(image_dir, image_name)
    predictions = []
    for model_path in model_paths:
        load_model(model, model_path)
        prediction = predict_image(model, image_path)
        predictions.append(prediction)
    results.append([image_name] + predictions)

# 保存到Excel文件
df = pd.DataFrame(results, columns=['Name', 'd33', 'jiedian', 'C33'])
output_path = r"E:\picture_diffusion_vae1\parat\predicted_results540_320_1.33_0.208.xlsx"
df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

Results saved to E:\picture_diffusion_vae1\parat\predicted_results540_320_1.33_0.208.xlsx
